ticket_master.ipynb

In [31]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
# timdelta
from datetime import datetime, timedelta

load_dotenv()

True

In [ ]:
def get_sports_events_df(city="San Diego"):
    """Get San Diego sports events with available tickets for next 6 months"""
    
    # get api key from .env
    api_key = os.getenv("ticketmaster_api_key")
    
    # set date range
    start_date = datetime.now()
    end_date = start_date + timedelta(days=180)  # 6 months
    # format dates for api
    start_str = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_str = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    # endpoint
    url = "https://app.ticketmaster.com/discovery/v2/events.json"
    
    # params - specifically for sports events
    params = {
        "apikey": api_key,
        "size": 500,  # increased size to get more events
        "city": city, 
        "startDateTime": start_str,
        "endDateTime": end_str,
        "segmentName": "Sports",  # Filter for sports events
        "sort": "date,asc",
        "includeTBA": "no",  # Exclude TBA events
        "includeTBD": "no"   # Exclude TBD events
    }
    
    try:
        # make api request
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Check if we have events in the response
        if '_embedded' not in data or 'events' not in data['_embedded']:
            return pd.DataFrame()  # Return empty DataFrame if no events
            
        # Extract events
        events = data['_embedded']['events']
        
        # Create lists to store our event data
        event_data = []
        
        for event in events:
            # Only include events that are on sale
            status = event.get('dates', {}).get('status', {}).get('code')
            if status != 'onsale':
                continue
                
            # Extract venue info safely
            venue_info = event.get('_embedded', {}).get('venues', [{}])[0]
            
            # Get more specific sports info
            classifications = event.get('classifications', [{}])[0]
            sport_type = classifications.get('subGenre', {}).get('name') or classifications.get('genre', {}).get('name')
            
            # Create a dictionary for each event
            event_dict = {
                'name': event.get('name'),
                'date': event.get('dates', {}).get('start', {}).get('localDate'),
                'time': event.get('dates', {}).get('start', {}).get('localTime'),
                'venue': venue_info.get('name'),
                'sport_type': sport_type,
                'min_price': None,
                'max_price': None,
                'tickets_url': event.get('url')
            }
            
            # Add price range if available
            if 'priceRanges' in event:
                event_dict['min_price'] = event['priceRanges'][0].get('min')
                event_dict['max_price'] = event['priceRanges'][0].get('max')
                
            event_data.append(event_dict)

            # filter out 'Petco Park Tours'
            event_data = [event for event in event_data if 'Tours' not in event['name']]
        
        # Create DataFrame
        df = pd.DataFrame(event_data)
        
        # Clean up the data
        if not df.empty and 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
        
        # Sort by date
        if not df.empty:
            df = df.sort_values('date')
        
        return df
        
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err.response.status_code}")
        return pd.DataFrame()
    except Exception as err:
        print(f"Error occurred: {err}")
        return pd.DataFrame()

if __name__ == "__main__":
    # Get sports events DataFrame
    sports_df = get_sports_events_df("San Diego")
    
    if not sports_df.empty:
        print(f"\nFound {len(sports_df)} upcoming sports events in San Diego:")
        display(sports_df)
    else:
        print("No events or error occurred.")


Found 48 upcoming sports events in San Diego:


,name,date,time,venue,sport_type,min_price,max_price,tickets_url
0,DIRECTV Holiday Bowl Washington State Cougars ...,2024-12-27,17:00:00,Snapdragon Stadium,College,50.0,380.0,https://www.ticketmaster.com/directv-holiday-b...
1,San Diego Gulls vs. Ontario Reign,2024-12-27,19:00:00,Pechanga Arena San Diego,Minor League,NaN,NaN,https://www.ticketmaster.com/event/Z7r9jZ1A7utoS
2,San Diego State Men's Basketball vs. Utah Stat...,2024-12-28,15:00:00,Viejas Arena at Aztec Bowl San Diego State Uni...,College,28.0,206.0,https://www.ticketmaster.com/san-diego-state-m...
3,San Diego Gulls vs. Bakersfield Condors,2024-12-28,18:00:00,Pechanga Arena San Diego,Minor League,NaN,NaN,https://www.ticketmaster.com/event/Z7r9jZ1A7utoV
4,San Diego State Aztecs Women's Basketball vs. ...,2024-12-29,13:00:00,Viejas Arena at Aztec Bowl San Diego State Uni...,College,6.0,20.0,https://www.ticketmaster.com/san-diego-state-a...
5,San Diego Gulls vs. San Jose Barracuda,2025-01-01,19:00:00,Pechanga Arena San Diego,Minor League,NaN,NaN,https://www.ticketmaster.com/event/Z7r9jZ1A7uto4
6,San Diego State Aztecs Women's Basketball vs. ...,2025-01-04,13:00:00,Viejas Arena at Aztec Bowl San Diego State Uni...,College,6.0,20.0,https://www.ticketmaster.com/san-diego-state-a...
7,Monster Jam,2025-01-04,17:00:00,Snapdragon Stadium,Motorsports/Racing,25.0,155.0,https://www.ticketmaster.com/monster-jam-san-d...
8,San Diego Seals vs. Ottawa Black Bears,2025-01-04,19:00:00,Pechanga Arena San Diego,Miscellaneous,NaN,NaN,https://www.ticketmaster.com/event/Z7r9jZ1A78_OE
9,Monster Jam,2025-01-05,15:00:00,Snapdragon Stadium,Motorsports/Racing,25.0,130.0,https://www.ticketmaster.com/monster-jam-san-d...
